In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("24121002_SparkSQL_execution_plan").getOrCreate()

24/12/10 15:28:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [24]:
df= spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/emp.csv')

In [25]:
df.show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
| 9292|  JACK|

In [35]:
dept_df= spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/dept.csv')

In [36]:
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [37]:
df.createOrReplaceTempView('emp_df_view')

In [38]:
dept_df.createOrReplaceTempView('dept_df_view')

In [8]:
#부에서 조인 
join_query='''
select *
from emp_df_view join dept_df_view on emp_df_view.deptno=dept_df_view.deptno
'''

#??

In [9]:
viewEmpDept= spark.sql(join_query)
viewEmpDept.show() #하나의 테이블로 합

+-----+-----+--------+----+----------+----+----+------+-----+------+--------+----+----------+----+----+------+
|empno|ename|     job| mgr|  hiredate| sal|comm|deptno|empno| ename|     job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+--------+----+----------+----+----+------+-----+------+--------+----+----------+----+----+------+
| 7369|SMITH|   CLERK|7902|1980-12-17| 800|null|    20| 7902|  FORD| ANALYST|7566|1981-12-03|3000|null|    20|
| 7369|SMITH|   CLERK|7902|1980-12-17| 800|null|    20| 7876| ADAMS|   CLERK|7788|1987-05-23|1100|null|    20|
| 7369|SMITH|   CLERK|7902|1980-12-17| 800|null|    20| 7788| SCOTT| ANALYST|7566|1987-04-19|3000|null|    20|
| 7369|SMITH|   CLERK|7902|1980-12-17| 800|null|    20| 7566| JONES| MANAGER|7839|1981-04-02|2975|null|    20|
| 7369|SMITH|   CLERK|7902|1980-12-17| 800|null|    20| 7369| SMITH|   CLERK|7902|1980-12-17| 800|null|    20|
| 7499|ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30| 7900| JAMES|   CLERK|7698|1981-12-03| 950|null|    30|
|

In [29]:
viewEmpDept.createOrReplaceTempView('viewEmpDept') #등록 
#??

In [31]:
#부서위치가 'NEW YORK '인 직원 목록 
v_query='''
select *
from dept_df_view
where loc= 'NEW YORK'
'''

In [39]:
spark.sql('''

select * from
dept_df_view
''').show()

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [40]:
spark.sql(v_query).show() 

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
+------+----------+--------+



In [41]:
# subquery

q2= '''
select * 
from emp_df_view
where deptno =( 
 select deptno 
 from dept_df_view
 where loc= 'NEW YORK'
 )
'''

In [42]:
#case 2
spark.sql(q2).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
+-----+------+---------+----+----------+----+----+------+



In [43]:
#case1 의 실행계획 -join
spark.sql(v_query).explain()

== Physical Plan ==
*(1) Filter (isnotnull(loc#426) AND (loc#426 = NEW YORK))
+- FileScan csv [deptno#424,dname#425,loc#426] Batched: false, DataFilters: [isnotnull(loc#426), (loc#426 = NEW YORK)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab10/git_project/src/data/dept.csv], PartitionFilters: [], PushedFilters: [IsNotNull(loc), EqualTo(loc,NEW YORK)], ReadSchema: struct<deptno:int,dname:string,loc:string>




In [44]:
#case2 의 실행계획 서브쿼리 
spark.sql(q2).explain()

== Physical Plan ==
*(1) Filter (isnotnull(deptno#228) AND (deptno#228 = Subquery scalar-subquery#531, [id=#296]))
:  +- Subquery scalar-subquery#531, [id=#296]
:     +- *(1) Project [deptno#424]
:        +- *(1) Filter (isnotnull(loc#426) AND (loc#426 = NEW YORK))
:           +- FileScan csv [deptno#424,loc#426] Batched: false, DataFilters: [isnotnull(loc#426), (loc#426 = NEW YORK)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab10/git_project/src/data/dept.csv], PartitionFilters: [], PushedFilters: [IsNotNull(loc), EqualTo(loc,NEW YORK)], ReadSchema: struct<deptno:int,loc:string>
+- FileScan csv [empno#221,ename#222,job#223,mgr#224,hiredate#225,sal#226,comm#227,deptno#228] Batched: false, DataFilters: [isnotnull(deptno#228)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab10/git_project/src/data/emp.csv], PartitionFilters: [], PushedFilters: [IsNotNull(deptno)], ReadSchema: struct<empno:int,ename:string,job:string,mgr:int,hiredate:string,sal:int,comm:int,deptno:int>


In [45]:
spark.stop()